In [1]:
import tensorflow as tf 
import numpy as np 
import pandas as pd
from TFData.Dataset import Dataset
from model.models import ModelCreation

In [8]:
IMAGE_SIZE = [1024,1024]
IMAGE_SIZE_DESIRED = [512,512]
DATASET_SIZE = 33127
BATCH_SIZE = 32
EPOCHS = 5 

In [3]:
training_dataset = tf.io.gfile.glob('./Dataset/tfrecords/train*.tfrec')
test_dataset = tf.io.gfile.glob('./Dataset/tfrecords/test*.tfrec')

In [4]:
DataGenerator = Dataset(train_files=training_dataset,
                        test_files=test_dataset,
                        validation_split=0.2,
                        image_size=IMAGE_SIZE,
                        dataset_size=DATASET_SIZE,
                        batch_size=BATCH_SIZE)

In [5]:
# Here if you want to create training and validation dataset you must invoke get_train_and_validation_dataset
# ... at least I hope so 
DataGenerator.get_train_and_validation_dataset()

In [11]:
Model = ModelCreation(architecture='nasnet',
                      learning_rate= 0.0075,
                      input_shape=(*IMAGE_SIZE_DESIRED,3),
                      output_shape=1,
                      linear=None,
                      verbose=True)

AttributeError: 'Model' object has no attribute 'shape'

In [ ]:
Model.fit(
    DataGenerator.train_tfdataset,
    steps_per_epoch = DataGenerator.get_train_steps_per_epoch(),
    epochs = 5,
    validation_data = DataGenerator.validation_tfdataset,
    validation_steps = DataGenerator.get_validation_steps_per_epoch(),    
)